Course: DevOps, Maintenance, and Evolution

# Session #10: Scaling 






# The state of your projects?

In [1]:
from IPython.display import IFrame

url = 'http://64.225.103.230/commit_activity_daily.svg'
IFrame(url, width='100%', height='600') 

In [2]:
from IPython.display import IFrame

url = 'http://142.93.104.18/chart.svg'
IFrame(url, width='100%', height=500)

In [3]:
from IPython.display import IFrame

url = 'http://142.93.104.18/error_chart.svg'
IFrame(url, width='100%', height=500) 

## How is the SLA monitoring going?

- Which teams found their monitored teams to respect the SLAs? Partially?
- How is your team doing? Your monitored team? 
- What did you learn about writing SLAs?



## Pen-Testing 

- Did you succeed in finding vulnerabilities? Either your code or the other teams?
- Make sure to send your report to the tested group :)

## Logging
- Could you detect any attempts on your service?


# Bottlenecks appear in any good design

Most Common Reasons:
  * Congestion
  * Single point of failure

## Single Point of Failure

Very likely your systems look similar to the following:

![](images/possible_arch1.png)

## Single Point of Failure
 
Or perhaps you deployed the various components on their own machines:
![](images/possible_arch2.png)

## What happens if one of the nodes goes down?

- every one of the systems is a single point of failure
- the solution: replication

# Solution to Congestion - Scaling

Two Types of Scaling:
- Vertical
- Horizontal

## Vertical Scaling - Manually

  * In a physical server add more memory, harddisk etc.
  * Nice thing about using VMs, we can do that at runtime.

 
## Vertical Scaling with VirtualBox

Note the VM has to be powered off to perform modifications of RAM and storage. You can modify VMs via the GUI or on the CLI



### GUI

![](https://i.stack.imgur.com/IEtuL.png)

![](https://i.stack.imgur.com/AYfo9.png)

See https://askubuntu.com/a/510139

### CLI

To increase RAM of an existing VM to 8GB, storage to 64GB, and the CPU to more cores, one could run:

```bash
> VBoxManage list vms
"coursevm" {e072b310-1922-4113-93f5-2ca865e01722}
"lsd2018vm" {67fda2ea-7c3e-42f6-9a13-d0908020322d}
```

```bash
> VBoxManage modifyvm "coursevm" --cpus 8
> VBoxManage modifyvm "coursevm" --memory 8192
```

```bash
> VBoxManage list hdds
UUID:           d95b799f-0f33-4abd-925c-84ea5cbb3225
Parent UUID:    base
State:          created
Type:           normal (base)
Location:       /path/to/node1/ubuntu-16.04-amd64-disk001.vmdk
Storage format: VMDK
Capacity:       40960 MBytes
Encryption:     disabled

UUID:           9953ea1b-7295-4547-94fa-209f49c258f5
Parent UUID:    base
State:          created
Type:           normal (base)
Location:       /path/to/node1ubuntu-16.04-amd64-disk001.vmdk
Storage format: VMDK
Capacity:       40960 MBytes
Encryption:     disabled
```

```sh
> VBoxManage clonehd "9953ea1b-7295-4547-94fa-209f49c258f5" "cloned.vdi" --format vdi
> VBoxManage modifymedium disk "cloned.vdi" --resize 65536
```

Optionally, you may want to convert the disk back to `vmdk` with `VBoxManage clonehd "cloned.vdi" "resized.vmdk" --format vmdk`

**OBS** from within your VM make use of the new space by extending your disk with `gparted` or similar tools.

See:
  * Modfication of RAM https://www.virtualbox.org/manual/ch08.html#vboxmanage-modifyvm 
  * Modification of storage https://www.virtualbox.org/manual/ch08.html#vboxmanage-modifyvdi


## Vertical Scaling with DigitalOcean


### GUI


![](https://assets.nyc3.cdn.digitaloceanspaces.com/droplets/resize-options.png)



But -similar to VirtualBox- you can only resize VMs that are not running.

![](https://assets.nyc3.cdn.digitaloceanspaces.com/droplets/resize-switch-on.png)



See https://www.digitalocean.com/docs/droplets/how-to/resize/#resizing-via-the-control-panel



### CLI

Using the HTTP API vertical scaling could look as in the following.

See 

  * https://www.digitalocean.com/docs/droplets/how-to/resize/
  * https://developers.digitalocean.com/documentation/v2/#resize-a-droplet


**Note**: The following commands assume that you have defined the $DIGITAL_OCEAN_TOKEN envvar.


##### Create a New Droplet

```bash
> curl -X POST "https://api.digitalocean.com/v2/droplets" \
       -d'{"name":"New-Droplet","region":"fra1","size":"512mb","image":"ubuntu-16-04-x64"}' \
       -H "Authorization: Bearer $DIGITAL_OCEAN_TOKEN" \
	   -H "Content-Type: application/json"
```

##### List all Droplets

```sh
> curl -X GET "https://api.digitalocean.com/v2/droplets" -H "Authorization: Bearer $DIGITAL_OCEAN_TOKEN" | jq

{
  "droplets": [
    {
      "id": 116690982,
      "name": "webserver",
      "memory": 1024,
      "vcpus": 1,
      "disk": 30,
      "locked": false,
      "status": "active",
      "kernel": null,
      "created_at": "2018-10-26T12:54:08Z",
      "features": [],
...
      "networks": {
        "v4": [
          {
            "ip_address": "46.101.225.71",
            "netmask": "255.255.192.0",
            "gateway": "46.101.192.1",
            "type": "public"
          }
        ],
        "v6": []
      },
...
}
```


##### Resizing it

Resize CPU and RAM only to a 4GB droplet, which automatically shuts down the droplet. 

```sh
> curl -X POST -H 'Content-Type: application/json' \
       -H "Authorization: Bearer $DIGITAL_OCEAN_TOKEN" \
       -d '{"type":"resize","size":"s-2vcpu-4gb"}' \
       "https://api.digitalocean.com/v2/droplets/$DROPLET_ID/actions"
```

what's wrong with this?



##### Increase disk size:


```sh
> curl -X POST -H 'Content-Type: application/json' \
       -H "Authorization: Bearer $DIGITAL_OCEAN_TOKEN" \
       -d '{"type":"resize","size": "s-2vcpu-4gb","disk":true}'  \
       "https://api.digitalocean.com/v2/droplets/$DROPLET_ID/actions"
```


##### Power on

```sh
> curl -X POST -H 'Content-Type: application/json' \
       -H "Authorization: Bearer $DIGITAL_OCEAN_TOKEN" \
       -d '{"type":"power_on"}' \
       "https://api.digitalocean.com/v2/droplets/$DROPLET_ID/actions"
```



### Discussion

Now that is quite cool no? You could attach a resize action to an alarm and scale you machines vertically automatically.

But what is the drawback? You have to switch off the machines that you want to scale. That is quite unpractical if you have do not have reduntant machines, see more about that below.

# Horizontal Scaling


Horizontal scaling means not increasing the resources of a single (virtual) machine but instead adding more (virtual) machines to your cluster.

![](https://assets.digitalocean.com/blog/static/scaling-php/horizontal-scaling.png)

# Horizontal Scaling does not Only Solve Congestion but also Single Point of Failure

Note: probably change the title to match the next slide :)

![](images/possible_arch2.png)

# Redundancy/High-available Setup

![](https://assets.digitalocean.com/blog/static/floating-ips-start-architecting-your-applications-for-high-availability/ha-diagram-animated.gif)


For guides on manual horizontal scaling on DigitalOcean see for example:

  * https://blog.digitalocean.com/floating-ips-start-architecting-your-applications-for-high-availability/
  * https://www.digitalocean.com/community/tutorials/how-to-set-up-highly-available-web-servers-with-keepalived-and-floating-ips-on-ubuntu-14-04
  * https://www.digitalocean.com/community/tutorials/how-to-create-a-high-availability-setup-with-heartbeat-and-floating-ips-on-ubuntu-16-04
  * https://blog.digitalocean.com/load-balancers-simplifying-high-availability/
  * https://blog.digitalocean.com/horizontally-scaling-php-applications/

# Tools that Help Horizontal Scaling

  * Kubernetes, the elephant
  * Docker Swarm, the mouse


## Kubernetes

  * ... is popular in the US, see https://trends.google.com/trends/explore?geo=US&q=Kubernetes,Docker%20Swarm,Mesos
  * But since Danmark has another size, it is not that much more popular compared to alternatives, see
  https://trends.google.com/trends/explore?geo=DK&q=Kubernetes,Docker%20Swarm,Mesos


![](images/full-kubernetes-model-architecture.png)

## Docker Swarm

We are going for the mouse...

  * Based on `swarmkit`
    > A toolkit for orchestrating distributed systems at any scale
  * Orchestration
    - Manages nodes and services
  * Scheduling
    - Resource aware task scheduling
  
Scalability?
    

## Docker Swarm: Concepts

  * Nodes
    - A VM instance participating in a swarm 
    - Worker nodes
    - Manager nodes
  * Services and tasks
    - A collection of tasks to be executed on a node, i.e., containers
    - Replicated services
    - Global services (one instance per node)
  * Load balancing
See https://docs.docker.com/engine/swarm/

## Docker Swarm: Nodes

Two types of nodes: 
  * Managers
  * Workers

![](https://docs.docker.com/engine/swarm/images/swarm-diagram.png)



### Manager nodes

  * maintain cluster state
  * schedule services

For teaching we run a single swarm manager. In production you should have at least three manager nodes.

  * Three manager swarm tolerates a maximum loss of one manager.
  * Five manager swarm tolerates a maximum loss of two managers.
  * $n$ manager swarm tolerates maximum loss of $(n-1)/2$ managers.

Docker recommends a maximum of seven manager nodes for a swarm.

Important Note: Adding more managers does NOT mean increased scalability or higher performance. In general, the opposite is true.

### Worker nodes

  * are instances of Docker Engine whith sole purpose to execute containers
  * do not participate in scheduling decisions, or serve the swarm mode HTTP API
  * Have at least one manager node
  * (By default, all managers are also workers)

See more https://docs.docker.com/engine/swarm/how-swarm-mode-works/nodes/

![](https://docs.docker.com/engine/swarm/images/replicated-vs-global.png)

|Swarm Size|	Majority	|Fault Tolerance|
|:----------:|:-------------:|:------:|
|1	|1	|0|
|2	|2	|0|
|3	|2	|1|
|4	|3	|1|
|5	|3	|2|
|6	|4	|2|
|7	|4	|3|
|8	|5	|3|
|9	|5	|4|


https://docs.docker.com/engine/swarm/admin_guide/#add-manager-nodes-for-fault-tolerance

## New Docker Commands

  * `docker-machine` ... to manage virtual machines
    - usually used for quickly creating new VMs
  * `docker swarm` ... to manage a cluster (swarm)
  * `docker service` ... to manage replicated containers (services) in the swarm

### Locally Or on Other Providers than DO
If you want to create such a node on your local machine with VirtualBox as backend:

```sh
$ docker-machine create --driver virtualbox default
```

See more on that in the documentation: https://docs.docker.com/machine/get-started/


However, in the following we will use DigitalOcean for providing VMs for the nodes, see https://docs.docker.com/machine/drivers/digital-ocean/#usage

If your project runs on another provider you may want to check the other available drivers:
https://docs.docker.com/machine/drivers/


# Interactive: Docker Swarm cluster on DigitalOcean!

Create a Docker Swarm cluster on DigitalOcean (or locally with VirtualBox) with the following guide.

Note: 

- You need to have `docker-machine` installed. 
- define envvar: $DIGITAL_OCEAN_TOKEN


### Creating a Docker Swarm Cluster Node


```sh
> export DIGITALOCEAN_REGION=fra1 
```

```sh
> export DIGITALOCEAN_SIZE=1gb
```

```sh
> export DIGITALOCEAN_PRIVATE_NETWORKING=true
```

```sh
> docker-machine create --driver digitalocean \
                        --digitalocean-image ubuntu-18-04-x64 \
                        --digitalocean-access-token $DIGITAL_OCEAN_TOKEN \
                        node-0
```

... about 3min ... 

~~~bash
> docker-machine ls
NAME     ACTIVE   DRIVER         STATE     URL                         SWARM   DOCKER     ERRORS
node-0   -        digitalocean   Running   tcp://142.93.109.102:2376           v18.09.0
~~~



### Creating Two More Nodes

```bash
> docker-machine create --driver digitalocean \
                        --digitalocean-image ubuntu-18-04-x64 \
                        --digitalocean-access-token $DIGITAL_OCEAN_TOKEN \
                        node-1
```

```sh
> docker-machine create --driver digitalocean \
                        --digitalocean-image ubuntu-18-04-x64 \
                        --digitalocean-access-token $DIGITAL_OCEAN_TOKEN \
                        node-2
```




#### Alternatively, you could create a cluster with multiple nodes via a loop:


```bash
#!/bin/bash

export DIGITALOCEAN_REGION=fra1
export DIGITALOCEAN_SIZE=1gb
export DIGITALOCEAN_PRIVATE_NETWORKING=true
for (( i=0; i<=2; i++ ))
do
    echo "Creating node $i"
    docker-machine create --driver digitalocean \
                          --digitalocean-image ubuntu-18-04-x64 \
                          --digitalocean-access-token $DIGITAL_OCEAN_TOKEN \
                          node-$i
done
```



### Getting the IP of a Cluster Node

~~~bash
$ docker-machine ip node-0
~~~




### Making `node-0` a Cluster Manager

```bash
> SWARM_MANAGER_IP=`docker-machine ip node-0`
```

```sh
> echo $SWARM_MANAGER_IP
```

```
142.93.109.102
```


```sh

> docker-machine ssh node-0 "docker swarm init --advertise-addr $SWARM_MANAGER_IP"

```


```
Swarm initialized: current node (sozjy3nmfrieacm2pbgj41ek3) is now a manager.

To add a worker to this swarm, run the following command:

    docker swarm join --token SWMTKN-1-4rndqz4hwe38wtbl9fwgj33rk48ok3hri7a0xy42o7sf5ll38z-afkri2vu57m5z31v34bny16aj 142.93.109.102:2377

To add a manager to this swarm, run 'docker swarm join-token manager' and follow the instructions.
```



### Converting node-1 and node-2 to Workers

Now let's get that token from the remote node and build a command that we can run on node-1 and node-2 to join the swarm.

```sh
> docker-machine ssh node-0 "docker swarm join-token worker -q"
SWMTKN-1-4rndqz4hwe38wtbl9fwgj33rk48ok3hri7a0xy42o7sf5ll38z-afkri2vu57m5z31v34bny16aj

> MANAGER_TOKEN=`docker-machine ssh node-0 "docker swarm join-token worker -q"`

> REMOTE_CMD="docker swarm join --token $MANAGER_TOKEN $SWARM_MANAGER_IP:2377"

> docker-machine ssh node-1 "$REMOTE_CMD"
```

```
  This node joined a swarm as a worker.
```

```sh
> docker-machine ssh node-2 "$REMOTE_CMD"
```

```
  This node joined a swarm as a worker.
```






### On the manager, we can see the state of the cluster

```sh

> docker-machine ssh node-0 "docker node ls"

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS      ENGINE VERSION
sozjy3nmfrieacm2pbgj41ek3 *   node-0              Ready               Active              Leader              18.09.0
hy6ie5xq561f9w1zpiyaqkrk5     node-1              Ready               Active                                  18.09.0

```



### Starting a Service

Now that everything is setup, let's run a service on our cluster:

```sh
> docker-machine ssh node-0 "docker service create -p 8080:8080 --name appserver stifstof/crashserver"
overall progress: 0 out of 1 tasks
...
overall progress: 1 out of 1 tasks
verify: Waiting 5 seconds to verify that tasks are stable...
...
verify: Waiting 1 seconds to verify that tasks are stable...
verify: Service converged
```

... about 1-2 min ...





### Checking the State of the Service


```bash
> docker-machine ssh node-0 "docker service ls"
ID                  NAME                MODE                REPLICAS            IMAGE                        PORTS
ttkqm9wzthgu        appserver           replicated          1/1                 stifstof/crashserver:latest   *:8080->8080/tcp
```

You may directly ask for the state of a service with

```bash
> docker-machine ssh node-0 "docker service ps appserver"
```


Now, on a Mac you can: 

```sh
> open http://$SWARM_MANAGER_IP:8080
```

Alternatively, navigate manually to the swarm manager's IP port 8080 and see the webpage served. 



#### What happens if you navigate to the worker node ips? 


```sh
open http://`docker-machine ip node-1`:8080
```

```sh
open http://`docker-machine ip node-2`:8080
```

### The Routing Mesh


> The routing mesh enables each node in the swarm to accept connections on published ports for any service running in the swarm, even if there’s no task running on the node. The routing mesh routes all incoming requests to published ports on available nodes to an active container.


Read more:  https://docs.docker.com/engine/swarm/ingress/


# The Swarm also Restarts Services


To demonstrate this, we used the crashserver service - which is a webserver which kills itself three seconds after serving an http request.

Take some time and observe the behavior of the container before continuing with the guide. 
Note how the infrastructure is self-healing, by checking the state of the service multiple times after an invocation as shown above.

The service becomes unavailable while Swarm is recreating the container after it has been killed. 
Now we will scale the service to increase availability.





## Finally, Scaling

```bash
> docker-machine ssh node-0 "docker service scale appserver=5"
> docker-machine ssh node-0 "docker service ls"

ID                  NAME                MODE                REPLICAS            IMAGE                        PORTS
ttkqm9wzthgu        appserver           replicated          5/5                 stifstof/crashserver:latest   *:8080->8080/tcp



> docker-machine ssh node-0 "docker service ps appserver"


ID                  NAME                IMAGE                        NODE                DESIRED STATE       CURRENT STATE            ERROR               PORTS
vbg02o9bsaog        appserver.1         stifstof/crashserver:latest   node-1              Running             Running 7 minutes ago
mudpe1lokpj7        appserver.2         stifstof/crashserver:latest   node-0              Running             Running 13 seconds ago
t7enei6pz4jw        appserver.3         stifstof/crashserver:latest   node-0              Running             Running 12 seconds ago
sfpn4f2kg5nq        appserver.4         stifstof/crashserver:latest   node-1              Running             Running 39 seconds ago
wa8f99b6t199        appserver.5         stifstof/crashserver:latest   node-0              Running             Running 12 seconds ago
```




# Did it work? 

You should now be able to invoke the webpage without seeing the error-page each time the container is killed, but instead see the request being served by another container. Nice!

Althrough it is possible to kill all container by manically invoking the /status endpoint, if you want to test the self-healing feature of swarm, you can invoke the /kill endpoint, which will kill the container immediately, so you don't have to wait.  





# Cleaning up to not pay anymore...



```bash
> docker-machine ls --filter name=node-.* --filter driver=digitalocean --format "{{.Name}}" | xargs docker-machine rm -y

About to remove node-0, node-1
WARNING: This action will delete both local reference and remote instance.
Successfully removed node-0
Successfully removed node-1
```



The guide above is based on the guide at:
https://www.digitalocean.com/community/tutorials/how-to-create-a-cluster-of-docker-containers-with-docker-swarm-and-digitalocean-on-ubuntu-16-04

# Upgrade Strategies


  * _Blue-green_ ... Two identical environments, where only one is hot at any time
  ![](https://opensource.com/sites/default/files/f1_2.png)

  * _Canary_ ... Deploy to a small group first, then deploy to the rest
  ![](https://opensource.com/sites/default/files/f3_0.png)


  * _Rolling_ ... Deploy in rolling iterations
  ![](https://opensource.com/sites/default/files/images/business-uploads/rolling2.gif)
  
See https://opensource.com/article/17/5/colorful-deployments

## Docker Swarm: Rolling Updates

  1. Stop the first task
  2. Schedule update for the stopped task
  3. Start the container for the updated task
  4. If the update to a task returns RUNNING, wait for the specified delay period then start the next task
  5. If, at any  me during the update, a task returns FAILED, pause the update

PS: You need at least two replicas!


See https://docs.docker.com/engine/swarm/swarm-tutorial/rolling-update/

## Automating the Automation: Deploying the Swarm with Terraform

#### Terraform
- declarative scripting of complex cloud infrastructure
- like vagrant but for the cloud
- unlike Puppet, Chef which are for automating software on a single machine


# Your Task Until Next Week


Either 

  * create a high-available setup with _hot_ and _standby_ server for your MiniTwit, or
    - you don't need to replicate the load balancer too; suffices to replicate your server
  * create a Docker Swarm cluster for your MiniTwit in which all components run as services.
  
  
In both cases implement a rolling update strategy to your build chain.





### Stuff that came up during the chat / discussions:

- Talk about scalability problems due to too many services at Dell:  https://www.youtube.com/watch?v=gfh-VCTwMw8


- Floating IPs = Elastic IPs @ Amazon


- Information on the exam report? 
  - We'll post it in writing in the upcoming days


- Isn't it too much work to switch to container orchestration while bringing along monitoring and ...
  - It's up to you how much you want to go with this
  
In general:
- think about the cost implications & tradeoffs of various architectures  -- probably an interesting project / thesis by the way

Also: 
- Press F to pay respect: https://knowyourmeme.com/memes/press-f-to-pay-respects

